<a href="https://colab.research.google.com/github/maverickxlr/Koa_App/blob/main/Koa_App(revised).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from google.colab import drive
from keras.utils.image_utils import img_to_array
from skimage import io
from numpy import asarray
from PIL import Image

In [ ]:
!pip install streamlit 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!pip install --upgrade streamlit

In [ ]:
#!pip install pyngrok

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%writefile app1.py 
import streamlit as st
import cv2
import io
from PIL import Image
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras.utils.image_utils import img_to_array



map_dict1 = {0: 'Healthy knee image'}
map_dict2 = {0: 'Grade 1 (Doubtful)',
             1: 'Grade 2 (Minimal)',
             2: 'Grade 3 (Moderate)',
             3: 'Grade 4 (Severe)'}



def image_preprocessing(uploaded_file):
    image = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
    image = cv2.imdecode(image,1)
    image = Image.fromarray(image)        
    image = image.resize((224,224)) 
    image= img_to_array(image)
    image= np.expand_dims(image, axis=0)
    return image

@st.cache_resource
def load_binarymodel():
    binary_model = tf.keras.models.load_model('/content/drive/MyDrive/Saved_models/Resnet_binary/weights-improvement-36-0.94.hdf5')
    return binary_model

@st.cache_resource
def load_multiclassmodel():
    multiclass_model = tf.keras.models.load_model('/content/drive/MyDrive/Saved_models/Resnet_multiclass_onehotencoded/weights-improvement-26-1.00.hdf5.hdf5')
    return multiclass_model


#st.title("Knee Osteoarthritis Detector")
st.image("/content/drive/MyDrive/Screenshot logo 2.png")
st.text("This application predicts Knee Osteoarthritis severity by analyzing X-Ray images.")
st.sidebar.title("About This App")
st.sidebar.markdown("The application determines the severity of knee osteoarthritis on the basis of KL grade classification, by using machine learning algorithms.")
st.sidebar.title("Classification Grades")
st.sidebar.header("Grade 0 (none):")
st.sidebar.markdown("Definite absence of x-ray changes of osteoarthritis")
st.sidebar.header("Grade 1 (doubtful):")
st.sidebar.markdown("Doubtful joint space narrowing and possible osteophytic lipping")
st.sidebar.header("Grade 2 (minimal):")
st.sidebar.markdown("Definite osteophytes and possible joint space narrowing")
st.sidebar.header("Grade 3 (moderate):")
st.sidebar.markdown("Moderate multiple osteophytes, definite narrowing of joint space and some sclerosis and possible deformity of bone ends")
st.sidebar.header("Grade 4 (severe):")
st.sidebar.markdown("Large osteophytes, marked narrowing of joint space, severe sclerosis and definite deformity of bone ends")
#st.sidebar.image("/content/drive/MyDrive/The-KL-grading-system-to-assess-the-severity-of-knee-OA-Source.png",width=None,use_column_width='never')

### load file
uploaded_file = st.file_uploader("Please upload an X-Ray image", type=['png','jpeg','jpg'])
 
#picture = st.camera_input("Take a picture")

if uploaded_file is not None:
    st.image(uploaded_file, caption='Uploaded Image.')

#if picture is not None:
    #st.image(picture,caption='Captured Image')    

Generate_pred = st.button("Generate Prediction")   

if uploaded_file is not None:
   if Generate_pred:
      image=image_preprocessing(uploaded_file)

      binary_prediction = load_binarymodel().predict(image)
      binary_pred = (binary_prediction>=0.5).astype(int)
  
      if (binary_pred<0.5):
          st.title("Healthy knee image")
      else:
          multiclass_prediction = load_multiclassmodel().predict(image).argmax()
          st.title("Prediction: {}".format(map_dict2 [multiclass_prediction])) 

#if picture is not None:
   #if Generate_pred:
      #image=image_preprocessing(picture)

      #binary_prediction = load_binarymodel().predict(image)
      #binary_pred = (binary_prediction>=0.5).astype(int)
  
      #if (binary_pred<0.5):
          #st.title("Healthy knee image")
      #else:
          #multiclass_prediction = load_multiclassmodel().predict(image).argmax()
          #st.title("Prediction: {}".format(map_dict2 [multiclass_prediction])) 
          


Overwriting app1.py


In [ ]:
#from pyngrok import ngrok
#ngrok.set_auth_token("2OeFs65fVdnXWfT7l2qU6ZxZDvy_5tsPNwaGxvNMDHJkvRnk1")

In [ ]:
!streamlit run app1.py & npx localtunnel --port 8501


[#######...........] / extract:localtunnel: verb lock using /root/.npm/_locks/s

npx: installed 22 in 5.366s
your url is: https://silly-cloths-roll-34-74-202-167.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.74.202.167:8501

2023-05-01 08:37:51.412395: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
1/1 [==============================] - 1s 1s/step


In [ ]:
#!nohup streamlit run app.py --server.port 80  & 
#url = ngrok.connect()
#print(url) #generates our URL
#streamlit run --server.port 9095 app.py >/dev/null #used for starting our server